In [65]:
# For text preprocessing
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# For topic modeling
from gensim import corpora
from gensim.models import LdaModel

# Download NLTK Resources
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\naufa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\naufa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\naufa\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [67]:
documents = [
    "Rafael Nadal Joins Roger Federer in Missing U.S. Open",
    "Rafael Nadal Is Out of the Australian Open",
    "Biden Announces Virus Measures",
    "Biden's Virus Plans Meet Reality",
    "Where Biden's Virus Plan Stands"
]

In [69]:
import nltk

nltk.download('punkt_tab')
nltk.download('punkt', force=True)
nltk.download('stopwords', force=True)
nltk.download('wordnet', force=True)

# Initialize stopwords and lemmatizer
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Define preprocessing function
def preprocess_text(text):
    # Check input is a string
    if not isinstance(text, str):
        return []
    tokens = word_tokenize(text.lower())                     # Lowercase and tokenize
    tokens = [token for token in tokens if token.isalnum()]  # Remove punctuation
    tokens = [token for token in tokens if token not in stop_words]  # Remove stopwords
    tokens = [lemmatizer.lemmatize(token) for token in tokens]       # Lemmatize
    return tokens


# Preprocess all documents
preprocessed_documents = [preprocess_text(doc) for doc in documents]
preprocessed_documents


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\naufa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\naufa\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\naufa\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\naufa\AppData\Roaming\nltk_data...


[['rafael', 'nadal', 'join', 'roger', 'federer', 'missing', 'open'],
 ['rafael', 'nadal', 'australian', 'open'],
 ['biden', 'announces', 'virus', 'measure'],
 ['biden', 'virus', 'plan', 'meet', 'reality'],
 ['biden', 'virus', 'plan', 'stand']]

In [71]:
#Create a Gensim Dictionary object from the preprocessed documents
dictionary = corpora.Dictionary(preprocessed_documents)

#Convert each processed document into a bag-of-words representation using the dictionary
corpus = [dictionary.doc2bow(doc) for doc in preprocessed_documents]

In [73]:
#corpus: bag-of-words representation of the documents
#num_topics: number of topics to  be extracted by the model
#id2word=dictionary: dictionary mapping from IDs to words
#passes: number of passes through the corpus during training
# Train on LDA model on the corpus with 4 topics using Gensim's LdaModel class
lda_model = LdaModel(corpus, num_topics=2, id2word=dictionary, passes=15)

In [75]:
# empty list to store dominant topic labels for each document
article_labels = []

#iterate over each processed document
for i, doc in enumerate(preprocessed_documents):
    # for each document, convert to box representation
    bow = dictionary.doc2bow(doc)
    # get list of topic probabilities
    topics = lda_model.get_document_topics(bow)
    # determine topic with highest probability
    dominant_topic = max(topics, key=lambda x: x[1])[0]
    # appenf to the list
    article_labels.append(dominant_topic)

In [79]:
import pandas as pd

# Create DataFrame
df = pd.DataFrame({"Article": documents, "Topic": article_labels})

# Print the DataFrame
print("Table with Articles and Topic:")
print(df)
print()

Table with Articles and Topic:
                                             Article  Topic
0  Rafael Nadal Joins Roger Federer in Missing U....      1
1         Rafael Nadal Is Out of the Australian Open      1
2                     Biden Announces Virus Measures      0
3                   Biden's Virus Plans Meet Reality      0
4                    Where Biden's Virus Plan Stands      0



In [81]:
# Print the top terms for each topic
print("Top Terms for Each Topic:")
for idx, topic in lda_model.print_topics():
    print(f"Topic {idx}:")
    terms = [term.strip() for term in topic.split("+")]
    for term in terms:
        weight, word = term.split("*")
        print(f"-{word.strip()} (weight: {weight.strip()})")
        print()

Top Terms for Each Topic:
Topic 0:
-"biden" (weight: 0.166)

-"virus" (weight: 0.166)

-"plan" (weight: 0.119)

-"meet" (weight: 0.071)

-"reality" (weight: 0.071)

-"announces" (weight: 0.071)

-"measure" (weight: 0.071)

-"stand" (weight: 0.071)

-"australian" (weight: 0.024)

-"rafael" (weight: 0.024)

Topic 1:
-"nadal" (weight: 0.131)

-"open" (weight: 0.131)

-"rafael" (weight: 0.131)

-"missing" (weight: 0.079)

-"federer" (weight: 0.079)

-"join" (weight: 0.079)

-"roger" (weight: 0.079)

-"australian" (weight: 0.079)

-"virus" (weight: 0.027)

-"biden" (weight: 0.027)

